<a href="https://colab.research.google.com/github/ammobam/Display_SensorData/blob/main/210727_%EB%82%98%EB%AC%B4%ED%94%8C%EB%9E%98%EB%8B%9B_%EB%94%94%EC%8A%A4%ED%94%8C%EB%A0%88%EC%9D%B4_%EC%84%BC%EC%84%9C%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 불러오기
import pandas as pd
df_origin = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949')
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8145 entries, 0 to 8144
Columns: 841 entries, dat...date.name. to R.Vac
dtypes: float64(834), int64(6), object(1)
memory usage: 52.3+ MB


In [ ]:
# 컬럼명 리스트 만들기
col_origin = df_origin.columns
col_origin

'dat...date.name.'

In [ ]:
# 컬럼명 확인
col_origin[:-10]

Index(['dat...date.name.', 'LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101',
       'LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102',
       'LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103', 'LEHR.DRV.MTR..M4.1.SPD.1WSI40104',
       'LEHR.DRV.MTR..M4.2.SPD.1WSI40105', 'LEHR.DRV.MTR..M5.1.SPD.1WSI40106',
       'LEHR.DRV.MTR..M5.2.SPD.1WSI40107', 'LEHR.DRV.MTR..M6.1.SPD.1WSI40108',
       'LEHR.DRV.MTR..M6.2.SPD.1WSI40109',
       ...
       'STD_U_L', 'STD_U_R', 'STD_L_L', 'STD_L_R', 'STD_GLS_L', 'STD_GLS_R',
       '투입.LEFT.', '투입.RIGHT.', 'ROLLDOWN.폐기.LEFT.', '진공패드.폐기.LEFT.'],
      dtype='object', length=831)

In [ ]:
# 컬럼이름을 리스트로 변환
name_list = col_origin.tolist()

name_list_10=[]
for name in name_list:
    name = str(name)[:10]
    name_list_10.append(name)
name_list_10

In [ ]:
# 컬럼명(10글자)을 키, 각 컬럼의 인덱스를 값으로 하는 딕셔너리 생성
# 이렇게 하면 컬럼명 앞 10글자가 겹치는 컬럼의 인덱스를 모아 볼 수 있으므로
# 이름이 유사한 컬럼 집단의 개수, 각 집단에 속하는 컬럼의 수를 파악하기 쉬워짐

# 빈 딕셔너리 생성
name_dict = {}

# 인덱스, 컬럼이름에 대해 반복
for index, key in enumerate(name_list_10):
    
    # 만약 key가 딕셔너리에 없으면 키, 인덱스 생성
    if key not in name_dict:
        name_dict[key] = [index]
        # 딕셔너리에 넣을 때 int가 아닌 list로 넣어야 함
    # 만약 key가 딕셔너리에 있으면 인덱스 추가
    else :
        name_dict[key].append(index)


In [ ]:
name_dict

{'ATM.FLOW.S': [77,
  78,
  79,
  80,
  81,
  82,
  593,
  594,
  595,
  596,
  597,
  598,
  599,
  600,
  601,
  602,
  603,
  604,
  605,
  606,
  607,
  608,
  609,
  610,
  611,
  612],
 'ATM.N2.H2.': [624,
  625,
  626,
  627,
  628,
  629,
  630,
  631,
  632,
  633,
  634,
  635,
  636],
 'BATH.AMBIE': [413, 414],
 'CASHING.BL': [427],
 'CASHING.CO': [410],
 'CASHING.HO': [428],
 'DAMP.POSIT': [55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  415,
  416,
  417,
  418,
  419,
  420,
  421,
  422,
  423,
  424,
  425,
  426],
 'DB.BTM.DN.': [465, 466, 467, 468, 469],
 'DB.BTM.UP.': [462, 463, 464],
 'DB.HEAT.BO': [485, 486, 487, 488, 489, 490, 491],
 'DB.HEAT.BT': [460, 461],
 'DB.HEAT.TO': [446,
  447,
  448,
  449,
  450,
  451,
  452,
  453,
  454,
  455,
  456,
  457,
  458,
  459,
  478,
  479,
  480,
  481,
  482,
  483,
  484],
 'DB.N2.SCRE': [67, 68, 431, 432],
 'DCS_Offset': [46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  344,
  345,
  346,


In [ ]:
len(name_dict)

246